# Smart Terminal

`langchain`'s default `BashProcess` tool does not clean up Terminal formatting outputs. For example, cloning the langchain repo fails because `git clone` produces a lot of terminal output, which you normally wouldn't notice because it overwrites itself:

In [13]:
from langchain.agents import initialize_agent
from langchain.utilities.bash import BashProcess
from langchain.tools.shell.tool import ShellTool
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
tools = [ShellTool(process=BashProcess(persistent=True))]
agent = initialize_agent(tools, llm, verbose=True)
try:
    agent.run(
        "Clone the repo https://github.com/hwchase17/langchain . If it times out, then give up."
    )
except Exception as e:
    print(e)



> Entering new  chain...
 I need to clone the repo using the terminal.
Action: terminal
Action Input: git clone https://github.com/hwchase17/langchain
Observation: <3b5Cloning into 'langchain'...

remote: Enumerating objects: 40686, done.        
remote: Counting objects: 100% (11428/11428), done.        
remote: Compressing objects: 100% (1387/1387), done.        
remote: Total 40686 (delta 10520), reused 10423 (delta 10031), pack-reused 29258        
Receiving objects: 100% (40686/40686), 43.18 MiB | 5.94 MiB/s, done.
Resolving deltas: 100% (29583/29583), done.
Thought:This model's maximum context length is 4097 tokens, however you requested 8956 tokens (8700 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.


`langchain-contrib` offers a drop-in replacement terminal that condenses all of the output into a shorter version containing only the text that is visible to the user. Simply import `langchain_contrib.tools` and then load the `smart_terminal` tool instead:

In [16]:
from langchain_contrib.tools import load_tools

smart_agent = initialize_agent(load_tools(["smart_terminal"]), llm, verbose=True)
smart_agent.run(
    "Clone the repo https://github.com/hwchase17/langchain . If it times out, then give up."
)



> Entering new  chain...
 I need to clone the repo using a terminal command.
Action: Terminal
Action Input: git clone https://github.com/hwchase17/langchain
Observation: Cloning into 'langchain'...
remote: Enumerating objects: 43225, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 43225 (delta 89), reused 132 (delta 68), pack-reused 43039
Receiving objects: 100% (43225/43225), 46.53 MiB | 203.00 KiB/s, done.
Resolving deltas: 100% (30885/30885), done.
Thought: The repo has been successfully cloned.
Final Answer: The repo has been successfully cloned.

> Finished chain.


'The repo has been successfully cloned.'

Furthermore, the "smart terminal" only times out if there's no more output written to stdout, allowing you to still time out if the process hangs but not if it's simply taking a long time to process. Notice that on slower connections, regular langchain times out when running the clone command:

In [17]:
try:
    agent.run(
        "Clone the repo https://github.com/hwchase17/langchain . If it times out, then give up."
    )
except Exception as e:
    print(e)



> Entering new  chain...
 I need to clone the repo using the terminal.
Action: terminal
Action Input: git clone https://github.com/hwchase17/langchain
Observation: Timeout error while executing command git clone https://github.com/hwchase17/langchain
Thought: I can't clone the repo, so I should give up.
Final Answer: Give up.

> Finished chain.


As seen above, the contrib smart terminal successfully clones the repo even at slow speeds.

Finally, the smart terminal allows the LLM to access the environment variables you have already defined in your shell environment.

In [22]:
agent.run(
    "Get the value of the ZSH environment variable. "
    "If the output is empty, that means it doesn't exist."
)



> Entering new  chain...
 I need to find out what the value of the ZSH environment variable is.
Action: terminal
Action Input: echo $ZSH
Observation: 
Thought: I need to check if the variable exists.
Action: terminal
Action Input: env | grep ZSH
Observation: 
Thought: I now know the value of the ZSH environment variable.
Final Answer: The value of the ZSH environment variable is (the output of the command).

> Finished chain.


'The value of the ZSH environment variable is (the output of the command).'

Contrast with the smart terminal:

In [23]:
smart_agent.run(
    "Get the value of the ZSH environment variable. "
    "If the output is empty, that means it doesn't exist."
)



> Entering new  chain...
 I need to find out what the value of the ZSH environment variable is.
Action: Terminal
Action Input: echo $ZSH
Observation: /home/amos/.oh-my-zsh
Thought: I now know the final answer.
Final Answer: /home/amos/.oh-my-zsh

> Finished chain.


'/home/amos/.oh-my-zsh'